In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
from huggingface_hub import login
login(token="HF token")

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset
import re
import string

data = DatasetDict()

data = load_dataset("wnkh/MultiMed", 'Chinese')
source_language = 'Chinese'
target_language = 'Vietnamese'


In [ ]:
def normalize_text_to_chars(text):
    punctuation_to_remove_regex = f"[{re.escape(string.punctuation)}]"
    # Replace punctuation with a space
    text = re.sub(punctuation_to_remove_regex, " ", text).strip() if isinstance(text, str) else text
    # Make each character a word by inserting spaces
    return " ".join(text) if isinstance(text, str) else text

def map_to_multilingual_format(example):
    """
    Map the dataset fields to a multilingual translation format.
    """
    languages = ['English', 'Vietnamese', 'German', 'Chinese', 'French']
    
    # Create instruction for multilingual translation
    instruction = "Translate the following text into multiple languages."
    
    # Prepare input and outputs
    input_text = example['text']
    translations = {lang: example[lang] for lang in languages if lang in example}
    
    # Format the output as a structured response
    output = "\n".join([f"{lang}: {translations[lang]}" for lang in translations if translations[lang] is not None])
    
    return {
        "instruction": instruction,
        "input": input_text,
        "output": output
    }
    
for split in ['train', 'eval']:
    if 'Chinese' in raw_dataset[split].column_names:
        raw_dataset[split] = raw_dataset[split].map(
            lambda example: {'Chinese': normalize_text_to_chars(example['Chinese'])}
        )
    raw_dataset[split] = raw_dataset[split].map(
        lambda example: {'text': normalize_text_to_chars(example['text']) if isinstance(example['text'], str) else example['text']}
    )
    
data_train = raw_dataset["train"].map(map_to_multilingual_format)
data_eval = raw_dataset["eval"].map(map_to_multilingual_format)


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


In [ ]:
def formatting_prompts_func(examples):
    texts = []
    for instruction, input_text, output in zip(examples["instruction"], examples["input"], examples["output"]):
        if input_text and output:  # Only include non-empty examples
            text = alpaca_prompt.format(instruction, input_text, output)
            texts.append(text)
    return {"text": texts}

dataset = data_train.map(formatting_prompts_func, batched=True, remove_columns=data_train.column_names)
eval_dataset = data_eval.map(formatting_prompts_func, batched=True, remove_columns=data_eval.column_names)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 4,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,  # Increased batch size
        gradient_accumulation_steps = 2,  # Reduced for faster updates
        warmup_ratio = 0.01,  # Minimal warmup
        num_train_epochs = 1,  # Single epoch
        max_steps = 1000,  # Hard limit on training steps
        learning_rate = 2e-4,  # Increased learning rate
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 100,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",  # Simpler scheduler
        seed = 3407,
        output_dir = "",
        evaluation_strategy = "steps",
        eval_steps = 250,  # Less frequent evaluation
        save_strategy = "steps",
        save_steps = 500,  # Save only twice
        save_total_limit = 2,  # Keep only 2 checkpoints
        report_to = "wandb",
        gradient_checkpointing = True,
    ),
)

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()